In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx
from scipy.optimize import minimize
from sklearn.covariance import LedoitWolf


# Downloading historical prices

Both close and open prices are used. Trading is done at open based on close prices. The trading universe consists of the 50 largest publicly traded companies in Sweden. If yfinance messes around, just download from google colab and export to .csv file.

In [14]:

"""
tickers = [
    'INVE-B.ST', 'ATCO-B.ST', 'VOLV-B.ST', 'ASSA-B.ST', 'EQT.ST',
    'SEB-A.ST', 'ERIC-B.ST', 'HEXA-B.ST', 'SAND.ST', 'HM-B.ST',
    'SWED-A.ST', 'EPI-A.ST', 'SHB-A.ST', 'ESSITY-B.ST', 'ALFA.ST',
    'EVO.ST', 'LATO-B.ST', 'INDU-A.ST', 'LIFCO-B.ST',
    'LUND-B.ST', 'SAAB-B.ST', 'TELIA.ST', 'AKEL-D.ST',
    'SOBI.ST', 'INDT.ST', 'SCA-B.ST', 'SKF-B.ST', 'SKA-B.ST',
    'BEIJ-B.ST', 'ALIV-SDB.ST', 'SAVE.ST', 'HOLM-B.ST', 'AAK.ST',
    'SWEC-B.ST', 'SECT-B.ST', 'SSAB-B.ST', 'AXFO.ST', 'GETI-B.ST',
    'SAGA-B.ST', 'VOLCAR-B.ST', 'NDA-SE.ST', 'KINV-B.ST', 'BOL.ST',
    'FABG.ST', 'BALD-B.ST', 'CAST.ST', 'PEAB-B.ST', 'NCC-B.ST'
]

tickers = [
    'INVE-B.ST', 'ATCO-B.ST', 'VOLV-B.ST', 'ASSA-B.ST',
    'SEB-A.ST']
"""
# Analysis period, can easily be changed
start_date = "2010-01-01"
end_date = "2025-05-23"

# Download historical opening and closing prices
price_close = yf.download(tickers, start=start_date, end=end_date)['Close']
price_open = yf.download(tickers, start=start_date, end=end_date)['Open']


[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['ATCO-B.ST', 'INVE-B.ST']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['ATCO-B.ST', 'INVE-B.ST']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [16]:
### Downloads from csv files ###

price_close = pd.read_csv('price_close.csv')
price_open = pd.read_csv('price_open.csv')

In [4]:
# Simple code for handling stocks with insufficient amount of historical data, probably more than enough.   
def create_window_and_valid_tickers(returns, current_date):
    window = returns.loc[current_date - pd.DateOffset(years=4):current_date]
    valid_data = window.dropna(axis=1)  # remove stocks with any missing values
    return valid_data

"""
    GPT generated code for analyzing gaps in historical prices.
"""
# This function might be overkill
def analyze_return_gaps(price_df):
    summary = {}

    for ticker in price_df.columns:
        series = price_df[ticker]
        first_valid = series.first_valid_index()
        post_ipo_series = series.loc[first_valid:]
        missing = post_ipo_series.isna()

        gap_lengths = []
        current_gap = 0

        for val in missing:
            if val:
                current_gap += 1
            elif current_gap > 0:
                gap_lengths.append(current_gap)
                current_gap = 0

        # If gap at end
        if current_gap > 0:
            gap_lengths.append(current_gap)

        summary[ticker] = {
            'first_valid': first_valid,
            'total_days': len(post_ipo_series),
            'missing_days': missing.sum(),
            'missing_pct': 100 * missing.sum() / len(post_ipo_series),
            'gap_count': len(gap_lengths),
            'max_gap': max(gap_lengths) if gap_lengths else 0,
            'avg_gap': sum(gap_lengths) / len(gap_lengths) if gap_lengths else 0
        }

    return pd.DataFrame(summary).T.sort_values(by='missing_pct', ascending=False)


# Covariance matrix 

The next section handles estimation of covariance matrices used for the portfolio optimization.
Will include;  

1. Sample covariance with .cov() method.  

2. Sample covariance with Ledoit-Wolff shrinkage model from scikit-learn.  

3. EWMA based covariance matrix.  

When estimating covariance matrices, returns should be filtered so that they contain enough historical values without large gaps. For this implementation, 2 years of data will be used to estimate covariance matrices, stocks with >5% missing values will be dropped for EWMA.

In [ ]:
def sample_covariance(returns):

    return returns.cov()

In [ ]:
def ledoit_wolff_covariance(returns):
    
    model = LedoitWolf()
    cov_matrix = model.fit(returns).covariance_
    return cov_matrix



In [ ]:
def ewma_covariance(returns):
    
    lambda_ = 0.94
    # Initialize the covariance matrix (using the first row of returns)
    cov_matrix = returns.iloc[0].to_frame().dot(returns.iloc[0].to_frame().T)
    
    # Iteratively calculate EWMA covariance
    for t in range(1, len(returns)):
        r_t = returns.iloc[t].to_frame()
        cov_matrix = lambda_ * cov_matrix + (1 - lambda_) * r_t.dot(r_t.T)
    
    return cov_matrix.values



# Portfolio Optimization

In the next section, portfolio optimization models are implemented.  
Models include;  

1. Markowitz Mean-Variance  

2. Markowitz Minimum-Variance

3. ERC - Equal Risk Contributions  

4. HRP - Hiearchical Risk Parity   

In [ ]:
# Markowitz minimum variance

def markowitz_min_variance(returns, cov_func):

    n = returns.shape[1]
    sigma = cov_func(returns)
    w = cvx.Variable(n)
    portfolio_risk = cvx.quad_form(w, sigma)

    objective = cvx.Minimize(portfolio_risk)
    constraints = [cvx.sum(w) == 1, w >= 0]
    problem = cvx.Problem(objective, constraints)
    problem.solve(solver=cvx.ECOS)

    return pd.Series(w.value, index = returns.columns)



In [ ]:
# ERC - Equal Risk Contribution

def risk_contribution(weights, cov):
    portfolio_var = weights.T @ cov @ weights
    sigma = np.sqrt(portfolio_var)
    mrc = cov @ weights
    rc = weights * mrc
    return rc / sigma

def objective(weights, cov):
    rc = risk_contribution(weights, cov)
    target = np.mean(rc)
    return np.sum((rc - target)**2)

def equal_risk_portfolio(returns, cov_func):
    cov = cov_func(returns)
    n = cov.shape[0]
    x0 = np.ones(n) / n
    bounds = [(0, 1) for _ in range(n)]
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    result = minimize(objective, x0, args=(cov,), method='SLSQP', bounds=bounds, constraints=constraints)
    return pd.Series(result.x, index=returns.columns)

In [ ]:
# Hierarchical Risk Parity

#########################
#   Three stages:
#
#   1. Tree clustering
#   2. Quasi-diagonalization
#   3. Recursive bisection
#
#########################

# L1-Filter

L1-filter, as presented by X (20XX), is implemented.

In [ ]:
def l1_trend_filter(y, lam):
    n = len(y)
    x = cvx.Variable(n)
    objective = cvx.Minimize(0.5 * cvx.sum_squares(y - x) + lam * cvx.norm1(cvx.diff(x, 2)))
    prob = cvx.Problem(objective)
    prob.solve()
    return np.array(x.value)